<h1 id="tocheading">Spring 2018 NLP Class Project: Neural Machine Translation</h1>
<div id="toc"></div>

In [1]:
%%javascript
$.getScript('https://kmahelona.github.io/ipython_notebook_goodies/ipython_notebook_toc.js')

<IPython.core.display.Javascript object>

In [39]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random
import pandas as pd
import spacy
import pdb
import os
from underthesea import word_tokenize
import jieba
import numpy as np

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
import torch.utils.data

# running on cpu
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = torch.device("cpu" if torch.cuda.is_available() else "cpu")

In [5]:
# ! pip install spacy && python -m spacy download en

## Part 0: Project Overview

The goal of this project is to build a neural machine translation system and experience how recent advances have made their way. Each team will build the following sequence of neural translation systems for two language pairs, __Vietnamese (Vi)→English (En)__ and __Chinese (Zh)→En__ (prepared corpora is be provided):

1. Recurrent neural network based encoder-decoder without attention
2. Recurrent neural network based encoder-decoder with attention
2. Replace the recurrent encoder with either convolutional or self-attention based encoder.
4. [Optional] Build either or both fully self-attention translation system or/and multilingual translation system.

## Part 1: Data Upload & Preprocessing

In [3]:
PAD_token = 0
SOS_token = 1 # start of sentence
UNK_token = 2 # 2 = unk
EOS_token = 3 # end of sentence


class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {PAD_token:"<PAD>",
                           SOS_token: "<SOS>",
                           UNK_token:"<UNK>", 
                           EOS_token: "<EOS>"}
        self.n_words = 2  # Count SOS and EOS

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

In [4]:
# Turn a Unicode string to plain ASCII, thanks to
# http://stackoverflow.com/a/518232/2809427
def unicodeToAscii(s):
    """About "NFC" and "NFD": 
    
    For each character, there are two normal forms: normal form C 
    and normal form D. Normal form D (NFD) is also known as canonical 
    decomposition, and translates each character into its decomposed form. 
    Normal form C (NFC) first applies a canonical decomposition, then composes 
    pre-combined characters again.
    
    About unicodedata.category: 
    
    Returns the general category assigned to the Unicode character 
    unichr as string."""
    
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )


In [5]:
# Trim
def normalizeString(s):
    s = unicodeToAscii(s.strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    return s

In [6]:
def readLangs(lang1, lang2, reverse=False,
             dataset="train"):
    
    """Takes as input;
    - lang1, lang2: either (vi, en) or (zh, en)
    - dataset: one of ("train","dev","test")"""
    
    print("Reading lines...")
    
    eos = [".","?","!","\n"]
    
    # Read the pretokenized lang1 file and split into lines
    lang1_lines = open("../data/tokens_and_preprocessing_em/pretokenized_data/iwslt-%s-%s-processed/%s.tok.%s" % (lang1, lang2, dataset, lang1), encoding="utf-8").\
        read().strip().split("\n")
        
    # Read the lang2 file and split into lines
    lang2_lines = open("../data/tokens_and_preprocessing_em/pretokenized_data/iwslt-%s-%s-processed/%s.tok.%s" % (lang1, lang2, dataset, lang2), encoding="utf-8").\
        read().strip().split("\n")
    
    # create sentence pairs (lists of length 2 that consist of string pairs)
    # e.g. ["And we &apos;re going to tell you some stories from the sea here in video .",
    #       "我们 将 用 一些 影片 来讲 讲述 一些 深海 海里 的 故事  "]
    # check if there are the same number of sentences in each set
    assert len(lang1_lines) == len(lang2_lines), "Two languages must have the same number of sentences. "+ str(len(lang1_lines)) + " sentences were passed for " + str(lang1) + "." + str(len(lang2_lines)) + " sentences were passed for " + str(lang2)+"."
    # normalize if not Chinese, Chinese normalization is already handeled
    if lang1 == "zh":
        lang1_lines = [s + "<EOS>" for s in lang1_lines]
    else:
        lang1_lines = [normalizeString(s).replace(".","<EOS>").\
                       replace("?","<EOS>").replace("!","<EOS>").replace("\n","<EOS>") for s in lang1_lines]
    lang2_lines = [normalizeString(s).replace(".","<EOS>").\
                       replace("?","<EOS>").replace("!","<EOS>").replace("\n","<EOS>") for s in lang2_lines]
    # construct pairs
    pair_ran = range(len(lang1_lines))
    pairs = [[lang1_lines[i]] + [lang2_lines[i]] for i in pair_ran]
    
#     # Split every line into pairs and normalize
#     pairs = [[normalizeString(s) for s in l.split('\t')] for l in lines]

    # Reverse pairs, make Lang instances
    if reverse:
        pairs = [list(reversed(p)) for p in pairs]
        input_lang = Lang(lang2)
        output_lang = Lang(lang1)
    else:
        input_lang = Lang(lang1)
        output_lang = Lang(lang2)

    return input_lang, output_lang, pairs

In [7]:
def prepareData(lang1, lang2, reverse=False, dataset="train"):
    input_lang, output_lang, pairs = readLangs(lang1, lang2, reverse, dataset=dataset)
    print("Read %s sentence pairs" % len(pairs))
    print("Trimmed to %s sentence pairs" % len(pairs))
    print("Counting words...")
    for pair in pairs:
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])
    print("Counted words:")
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words)
    return input_lang, output_lang, pairs

# example
input_lang, output_lang, pairs = prepareData('vi', 'en', False, dataset="train")
print(random.choice(pairs))

Reading lines...
Read 133317 sentence pairs
Trimmed to 133317 sentence pairs
Counting words...
Counted words:
vi 16142
en 47566
[' e lam cho no ket noi voi nhau <EOS> e lam cho no co the lam cho moi nguoi muon hanh ong <EOS>', 'To make it connect <EOS> To make it make people want to act <EOS>']


In [12]:
input_lang, output_lang, pairs = prepareData('zh', 'en', False, dataset="train")
print(random.choice(pairs))

Reading lines...
Read 213376 sentence pairs
Trimmed to 213376 sentence pairs
Counting words...
Counted words:
zh 89202
en 59327
['但 我 相信 对 现今 今日 日趋 珍贵 的 自然 自然资源 资源 来说   它 会 是 一个 非常 巧妙 和 持续 持续性 的 辅助 物 <EOS>', 'But I do think it could be quite a smart and sustainable addition to our increasingly precious natural resources <EOS>']


### 1.1 Vietnamese to English

In [9]:
# # Please find the original tokenizing code provided by Elman Mansimov in the following link:
# # https://github.com/derincen/neural-machine-translation/tree/master/data/tokens_and_preprocessing_em/preprocess_translation

# def tokenize_vi(f_names, f_out_names):
#     for f_name, f_out_name in zip(f_names, f_out_names):
#         lines = open(f_name, 'r').readlines()
#         tok_lines = open(f_out_name, 'w')
#         for i, sentence in enumerate(lines):
#             if i > 0 and i % 1000 == 0:
#                 print (f_name.split('/')[-1], i, len(lines))
#             tok_lines.write(word_tokenize(sentence, format="text") + '\n')
#         tok_lines.close()

# def tokenize_en(f_names, f_out_names):
#     tokenizer = spacy.load('en_core_web_sm')

#     for f_name, f_out_name in zip(f_names, f_out_names):
#         lines = open(f_name, 'r').readlines()
#         tok_lines = open(f_out_name, 'w')
#         for i, sentence in enumerate(lines):
#             if i > 0 and i % 1000 == 0:
#                 print (f_name.split('/')[-1], i, len(lines))
#             # replaced tokenizer(sentence) with str(tokenizer(sentence)) to avoid 
#             # type error while joining
#             tok_lines.write(' '.join(str(tokenizer(sentence))) + '\n')
#         tok_lines.close()


# if __name__ == "__main__":
#     root = '../data/tokens_and_preprocessing_em/pretokenized_data/iwslt-vi-en-processed/'
#     tokenize_vi([os.path.join(root, 'train.vi'), os.path.join(root, 'dev.vi'), 
#                  os.path.join(root, 'test.vi')],\
#                [os.path.join(root, 'train.tok.vi'), os.path.join(root, 'dev.tok.vi'), 
#                 os.path.join(root, 'test.tok.vi')])

#     tokenize_en([os.path.join(root, 'train.en'), os.path.join(root, 'dev.en'), 
#                  os.path.join(root, 'test.en')],\
#                 [os.path.join(root, 'train.tok.en'), os.path.join(root, 'dev.tok.en'), 
#                  os.path.join(root, 'test.tok.en')])


In [8]:
# Format: languagepair_language_dataset
# Train 
vien_vi_train, vien_en_train, vi_en_train_pairs = prepareData('vi', 'en', False, dataset="train")
# Dev 
vien_vi_dev, vien_en_dev, vi_en_dev_pairs = prepareData('vi', 'en', False, dataset="dev")
# Test
vien_vi_test, vien_en_test, vi_en_test_pairs = prepareData('vi', 'en', False, dataset="test")

Reading lines...
Read 133317 sentence pairs
Trimmed to 133317 sentence pairs
Counting words...
Counted words:
vi 16142
en 47566
Reading lines...
Read 1268 sentence pairs
Trimmed to 1268 sentence pairs
Counting words...
Counted words:
vi 1368
en 3814
Reading lines...
Read 1553 sentence pairs
Trimmed to 1553 sentence pairs
Counting words...
Counted words:
vi 1323
en 3617


### 1.2 Chinese to English

In [11]:
# # Please find the original tokenizing code provided by Elman Mansimov in the following link:
# # https://github.com/derincen/neural-machine-translation/tree/master/data/tokens_and_preprocessing_em/preprocess_translation

# def tokenize_zh(f_names, f_out_names):
#     for f_name, f_out_name in zip(f_names, f_out_names):
#         lines = open(f_name, 'r').readlines()
#         tok_lines = open(f_out_name, 'w')
#         for i, sentence in enumerate(lines):
#             if i > 0 and i % 1000 == 0:
#                 print (f_name.split('/')[-1], i, len(lines))
#             tok_lines.write(' '.join(jieba.cut(sentence, cut_all=True)))
#         tok_lines.close()

# def tokenize_en(f_names, f_out_names):
#     tokenizer = spacy.load('en_core_web_sm')

#     for f_name, f_out_name in zip(f_names, f_out_names):
#         lines = open(f_name, 'r').readlines()
#         tok_lines = open(f_out_name, 'w')
#         for i, sentence in enumerate(lines):
#             if i > 0 and i % 1000 == 0:
#                 print (f_name.split('/')[-1], i, len(lines))
#             # replaced tokenizer(sentence) with str(tokenizer(sentence)) to avoid 
#             # type error while joining
#             tok_lines.write(' '.join(str(tokenizer(sentence))) + '\n')
#         tok_lines.close()

# if __name__ == "__main__":
#     root = '../data/tokens_and_preprocessing_em/pretokenized_data/iwslt-zh-en-processed/'
#     tokenize_zh([os.path.join(root, 'dev.zh'), os.path.join(root, 'test.zh'), os.path.join(root, 'train.zh')],\
#                 [os.path.join(root, 'dev.tok.zh'), os.path.join(root, 'test.tok.zh'), os.path.join(root, 'train.tok.zh')])

# #     tokenize_en([os.path.join(root, 'dev.en'), os.path.join(root, 'test.en'), os.path.join(root, 'train.en')],\
# #                [os.path.join(root, 'dev.tok.en'), os.path.join(root, 'test.tok.en'), os.path.join(root, 'train.tok.en')])


In [9]:
# Format: languagepair_language_dataset
# Train 
zhen_zh_train, zhen_en_train, zh_en_train_pairs = prepareData('zh', 'en', False, dataset="train")
# Dev 
zhen_zh_dev, zhen_en_dev, zh_en_dev_pairs = prepareData('zh', 'en', False, dataset="dev")
# Test
zhen_zh_test, zhen_en_test, zh_en_test_pairs = prepareData('zh', 'en', False, dataset="test")

Reading lines...
Read 213376 sentence pairs
Trimmed to 213376 sentence pairs
Counting words...
Counted words:
zh 89202
en 59327
Reading lines...
Read 1261 sentence pairs
Trimmed to 1261 sentence pairs
Counting words...
Counted words:
zh 6134
en 3914
Reading lines...
Read 1397 sentence pairs
Trimmed to 1397 sentence pairs
Counting words...
Counted words:
zh 5216
en 3421


In [10]:
zh_en_train_pairs[3]

['我们 将 用 一些 影片 来讲 讲述 一些 深海 海里 的 故事  <EOS>',
 'And we apos re going to tell you some stories from the sea here in video <EOS>']

### 1.3: Check Source & Target Vocabs

Since the source and target languages can have very different table lookup layers, it's good practice to have separate vocabularies for each. Thus, we build vocabularies for each language that we will be using. 

In the first class (Lang) of this section, we have already defined vocabularies for all languages. So, there is no need to redefine another function. We chech each vocabulary below.

#### Chinese Vocabulary

In [11]:
print ("The number of words in Chinese training corpus is " + str(zhen_zh_train.n_words))

The number of words in Chinese training corpus is 89202


In [12]:
zhen_zh_train.word2index["格"]

10481

In [13]:
zhen_zh_train.index2word[10481]

'格'

#### Vietnamese Vocabulary

In [14]:
print ("The number of words in Vietnamese training corpus is " + str(vien_vi_train.n_words))

The number of words in Vietnamese training corpus is 16142


In [15]:
vien_vi_train.word2index["Hamburger"]

6750

In [16]:
vien_vi_train.index2word[6750]

'Hamburger'

#### English Vocabulary for Zh-En

In [17]:
print ("The number of words in English training corpus for Zh-En is " + str(zhen_en_train.n_words))

The number of words in English training corpus for Zh-En is 59327


In [18]:
zhen_en_train.word2index["translate"]

1449

In [19]:
zhen_en_train.index2word[1451]

'directly'

#### English Vocabulary for Vi-En

In [20]:
print ("The number of words in English training corpus for Vi-En is " + str(vien_en_train.n_words))

The number of words in English training corpus for Vi-En is 47566


In [21]:
vien_en_train.word2index["machine"]

846

In [22]:
vien_en_train.index2word[846]

'machine'

### 1.4 Prepare Dataloaders

In [23]:
vien_en_dev.word2index["<EOS>"]

24

In [24]:
vien_en_dev.index2word[24]

'<EOS>'

In [26]:
PAD_IDX = 0
SOS_IDX = 1
UNK_IDX = 2
# EOS_IDX = 3
# convert token to id in the dataset
def token2index_dataset(paired_tokens, 
                        lang1_token2id_vocab,
                        lang2_token2id_vocab):
    """Takes as input:
    - paired_tokens: a list of sentence pairs that consist of source & target lang sentences.
    - lang1_token2id_vocab: token2index vocabulary for the first language. 
                            Get by method Lang_dataset.word2index
    - lang2_token2id_vocab: token2index vocabulary for the second language. 
                            Get by method Lang_dataset.word2index
                            
    Returns:
    - indices_data_lang_1, indices_data_lang2: A list of lists where each sub-list holds corresponding indices for each
                                               token in the sentence."""
    indices_data_lang_1, indices_data_lang_2 = [], []
    vocabs = [lang1_token2id_vocab, lang2_token2id_vocab]
    
    # lang1
    for t in range(len(paired_tokens)):
        # replaces token with UNK_IDX if the token is not in vocab
        index_list = [vocabs[0][token] if token in vocabs[0]\
                                    else UNK_IDX for token in paired_tokens[t][0]] 
        indices_data_lang_1.append(index_list)
    # lang2
    for t in range(len(paired_tokens)):
        index_list =  [vocabs[1][token] if token in vocabs[1] \
                                    else UNK_IDX for token in paired_tokens[t][1]] 
        indices_data_lang_2.append(index_list)
        
    return indices_data_lang_1, indices_data_lang_2

# train indices
zhen_zh_train_indices, zhen_en_train_indices = token2index_dataset(zh_en_train_pairs,
                                                                   zhen_zh_train.word2index,
                                                                   zhen_en_train.word2index)

vien_vi_train_indices, vien_en_train_indices = token2index_dataset(vi_en_train_pairs,
                                                                   vien_vi_train.word2index,
                                                                   vien_en_train.word2index)

# dev indices
zhen_zh_dev_indices, zhen_en_dev_indices = token2index_dataset(zh_en_dev_pairs,
                                                               zhen_zh_dev.word2index,
                                                               zhen_en_dev.word2index)

vien_vi_dev_indices, vien_en_dev_indices = token2index_dataset(vi_en_dev_pairs,
                                                               vien_vi_dev.word2index,
                                                               vien_en_dev.word2index)

# test indices
zhen_zh_test_indices, zhen_en_test_indices = token2index_dataset(zh_en_test_pairs,
                                                                 zhen_zh_test.word2index,
                                                                 zhen_en_test.word2index)

vien_vi_test_indices, vien_en_test_indices = token2index_dataset(vi_en_test_pairs,
                                                                 vien_vi_test.word2index,
                                                                 vien_en_test.word2index)

In [28]:
print(zhen_zh_train_indices[0])

[321, 7912, 2, 7912, 310, 2, 4, 2, 1586, 23701, 2, 2, 2, 275, 49581, 2, 2, 2, 5915, 6331, 2, 2, 5868, 16124, 5789, 2]


In [29]:
print(zhen_en_train_indices[0])

[4216, 16909, 49096, 8295, 2, 16909, 2107, 2, 268, 2, 8295, 2, 1735, 8295, 8295, 10558, 2, 1263, 28417, 8295, 158, 2107, 23]


In [30]:
# check length
# train
print ("Chinese training sentence count = "+str(len(zhen_zh_train_indices)))
print ("Chinese-English (En) training sentence count = "+str(len(zhen_en_train_indices)))
print ("\nVietnamese training sentence count = "+str(len(vien_vi_train_indices)))
print ("Vietnamese-English (En) training sentence count = "+str(len(vien_en_train_indices)))
# dev
print ("\nChinese dev sentence count = "+str(len(zhen_zh_dev_indices)))
print ("Chinese-English (En) dev sentence count = "+str(len(zhen_en_dev_indices)))
print ("\nVietnamese dev sentence count = "+str(len(vien_vi_dev_indices)))
print ("Vietnamese-English (En) dev sentence count = "+str(len(vien_en_dev_indices)))
# test
print ("\nChinese test sentence count = "+str(len(zhen_zh_test_indices)))
print ("Chinese-English (En) test sentence count = "+str(len(zhen_en_test_indices)))
print ("\nVietnamese test sentence count = "+str(len(vien_vi_test_indices)))
print ("Vietnamese-English (En) test sentence count = "+str(len(vien_en_test_indices)))

Chinese training sentence count = 213376
Chinese-English (En) training sentence count = 213376

Vietnamese training sentence count = 133317
Vietnamese-English (En) training sentence count = 133317

Chinese dev sentence count = 1261
Chinese-English (En) dev sentence count = 1261

Vietnamese dev sentence count = 1268
Vietnamese-English (En) dev sentence count = 1268

Chinese test sentence count = 1397
Chinese-English (En) test sentence count = 1397

Vietnamese test sentence count = 1553
Vietnamese-English (En) test sentence count = 1553


#### Dataloader

In [35]:
from torch.utils.data import Dataset

In [42]:
## TODO 

MAX_SENTENCE_LENGTH = 15
BATCH_SIZE = 32

# zhen token2index vocabs
zhen_zh_train_token2id = zhen_zh_train.word2index
zhen_en_train_token2id = zhen_en_train.word2index

# vien token2index vocabs
vien_vi_train_token2id = vien_vi_train.word2index
vien_en_train_token2id = vien_en_train.word2index

class TranslationDataset():
    """
    Class that represents a train/dev/test dataset that's readable for PyTorch
    Note that this class inherits torch.utils.data.Dataset
    """

    def __init__(self, 
                 data_source, # training indices data of the source language
                 data_target, # training indices data of the target language
                 token2id_source=None, # token2id dict of the source language
                 token2id_target=None  # token2id dict of the target language
                ):
        """
        @param data_list: list of character
        @param target_list: list of targets

        """
        self.source_sentences = data_source
        self.target_sentences = data_target
        print("self.source_sentences = " + str(self.source_sentences[0]))
        print("self.target_sentences = " + str(self.target_sentences[0]))
        
        self.token2id_source = token2id_source
        self.token2id_target = token2id_target
        # prints the mandarin token -> # dictionary
        """
        It also contains wrong tokenized words & Latin Alphabet words too
        '11<EOS>': 13275, '为啥': 13276, '指责': 13277, '超载': 13278, 
        '复杂度': 13279, '小玩意': 13280, '小玩意儿': 13281, '此行': 13282, 
        '断面': 13283, '强制': 13284, '制发': 13285, '花瓶': 13286,
        '糖': 13287, '年缴': 13288, '缴纳': 13289, '会费': 13290,
        '99': 13291, 'Photoshop': 13292, '4000<EOS>': 13293, 
        '升级': 13294, '佯': 13295, '谬': 13296, 'Microsoft': 13297, 
        'Word': 13298, '文字': 13299,
        """
#         print("self.token2id_source = " + str(self.token2id_source))
        # prints the english token -> # dictionary
#         print("self.token2id_target = " + str(self.token2id_target))        

    def __len__(self):
        return len(self.source_sentences)

    def __getitem__(self, batch_index):

#         source_word_idx, target_word_idx = [], []
        source_mask, target_mask = [], []
        
        for index in self.source_sentences[batch_index][:MAX_SENTENCE_LENGTH]:
            if index != UNK_IDX:
                source_mask.append(0)
            else:
                source_mask.append(1)
                
        for index in self.target_sentences[batch_index][:MAX_SENTENCE_LENGTH]:
            if index != UNK_IDX:
                target_mask.append(0)
            else:
                target_mask.append(1)
        
        source_indices = self.source_sentences[batch_index][:MAX_SENTENCE_LENGTH]
        target_indices = self.target_sentences[batch_index][:MAX_SENTENCE_LENGTH]
        
        source_list = [source_indices, source_mask, len(source_indices)]
        target_list = [target_indices, target_mask, len(target_indices)]
        
        return source_list + target_list

    
def translation_collate(batch, max_sentence_length):
    """
    Customized function for DataLoader that dynamically pads the 
    batch so that all data have the same length
    """
    source_data, target_data = [], []
    source_mask, target_mask = [], []
    source_lengths, target_lengths = [], []

    for datum in batch:
        source_lengths.append(datum[2])
        target_lengths.append(datum[5])
        
        # PAD
        source_data_padded = np.pad(np.array(datum[0]), 
                                    pad_width=((0, MAX_SENTENCE_LENGTH-datum[2])),
                                mode="constant", 
                                    constant_values=0)
        source_data.append(source_data_padded)
        
        source_mask_padded = np.pad(np.array(datum[1]), 
                                    pad_width=((0, MAX_SENTENCE_LENGTH-datum[2])),
                                mode="constant", 
                                    constant_values=0)
        source_mask.append(source_mask_padded)
        
        target_data_padded = np.pad(np.array(datum[3]), pad_width=((0, MAX_SENTENCE_LENGTH-datum[5])),
                                mode="constant", constant_values=0)
        target_data.append(target_data_padded)
        
        target_mask_padded = np.pad(np.array(datum[4]), pad_width=((0, MAX_SENTENCE_LENGTH-datum[5])),
                               mode="constant", constant_values=0)
        target_mask.append(target_mask_padded)
        
    ind_dec_order = np.argsort(source_lengths)[::-1]
    source_data = np.array(source_data)[ind_dec_order]
    target_data = np.array(target_data)[ind_dec_order]
    source_mask = np.array(source_mask)[ind_dec_order].reshape(len(batch), -1, 1)
    target_mask = np.array(target_mask)[ind_dec_order].reshape(len(batch), -1, 1)
    source_lengths = np.array(source_lengths)[ind_dec_order]
    target_lengths = np.array(target_lengths)[ind_dec_order]
    
    source_list = [torch.from_numpy(source_data), 
               torch.from_numpy(source_mask).float(), source_lengths]
    target_list = [torch.from_numpy(target_data), 
               torch.from_numpy(target_mask).float(), target_lengths]
        
    return source_list + target_list

print("Loading zhen_train_dataset:")
zhen_train_dataset = TranslationDataset(zhen_zh_train_indices,
                                       zhen_en_train_indices,
                                       token2id_source=zhen_zh_train_token2id,
                                       token2id_target=zhen_en_train_token2id)

zhen_train_loader = torch.utils.data.DataLoader(dataset=zhen_train_dataset,
                               batch_size=BATCH_SIZE,
                               collate_fn=lambda x, max_sentence_length=MAX_SENTENCE_LENGTH: translation_collate(x, MAX_SENTENCE_LENGTH),
                               shuffle=False)

print("\nLoading zhen_dev_dataset:")
zhen_dev_dataset = TranslationDataset(zhen_zh_dev_indices,
                                       zhen_en_dev_indices,
                                       token2id_source=zhen_zh_train_token2id,
                                       token2id_target=zhen_en_train_token2id)

zhen_dev_loader = torch.utils.data.DataLoader(dataset=zhen_dev_dataset,
                             batch_size=BATCH_SIZE,
                             collate_fn=lambda x, max_sentence_length=MAX_SENTENCE_LENGTH: translation_collate(x, MAX_SENTENCE_LENGTH),
                             shuffle=False)

print("\nLoading vien_train_dataset:")
vien_train_dataset = TranslationDataset(vien_vi_train_indices,
                                       vien_en_train_indices,
                                       token2id_source=vien_vi_train_token2id,
                                       token2id_target=vien_en_train_token2id)

vien_train_loader = torch.utils.data.DataLoader(dataset=vien_train_dataset,
                             batch_size=BATCH_SIZE,
                             collate_fn=lambda x, max_sentence_length=MAX_SENTENCE_LENGTH: translation_collate(x, MAX_SENTENCE_LENGTH),
                             shuffle=False)

print("\nLoading vien_dev_dataset:")
vien_dev_dataset = TranslationDataset(vien_vi_dev_indices,
                                       vien_en_dev_indices,
                                       token2id_source=vien_vi_train_token2id,
                                       token2id_target=vien_en_train_token2id)

vien_dev_loader = torch.utils.data.DataLoader(dataset=vien_dev_dataset,
                             batch_size=BATCH_SIZE,
                             collate_fn=lambda x, max_sentence_length=MAX_SENTENCE_LENGTH: translation_collate(x, MAX_SENTENCE_LENGTH),
                             shuffle=False)


Loading zhen_train_dataset:
self.source_sentences = [321, 7912, 2, 7912, 310, 2, 4, 2, 1586, 23701, 2, 2, 2, 275, 49581, 2, 2, 2, 5915, 6331, 2, 2, 5868, 16124, 5789, 2]
self.target_sentences = [4216, 16909, 49096, 8295, 2, 16909, 2107, 2, 268, 2, 8295, 2, 1735, 8295, 8295, 10558, 2, 1263, 28417, 8295, 158, 2107, 23]

Loading zhen_dev_dataset:
self.source_sentences = [2, 2, 1232, 1232, 2, 4, 2, 55, 91, 2, 2, 2, 2, 2695, 650, 2, 650, 14, 2, 766, 678, 2, 4782, 2, 2, 2, 281, 2, 2, 2, 27, 2802, 2, 430, 729, 2, 14, 2, 2, 2, 2, 16, 2, 4733, 5952, 2, 2, 2, 2, 2, 2, 2, 2]
self.target_sentences = [2, 2, 2, 2, 2, 3, 2, 2, 30, 199, 2, 3, 2, 2, 2, 419, 2, 419, 2, 2, 2, 2, 2, 30, 2, 1211, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 419, 2, 2, 2, 1211, 2, 2, 2, 51, 2, 2, 51, 2, 2, 2, 199, 2, 2, 2, 1109, 2, 2, 2, 2, 2, 2, 2, 2, 1211, 2, 2, 419, 2, 2, 2, 2, 2, 199, 2, 2, 2, 2, 2, 2, 2]

Loading vien_train_dataset:
self.source_sentences = [3018, 2843, 137, 44, 2, 2843, 137, 1836, 2, 44, 1830, 5219, 2, 1082, 44, 888,

In [43]:
##########################################
#### SAMPLE DATASET - CLEAR OUT LATER ####
##########################################

# zhen_train_dataset = TranslationDataset(zhen_zh_train_indices[:480], # 15 batches
#                                        zhen_en_train_indices[:480],
#                                        token2id_source=zhen_zh_train_token2id,
#                                        token2id_target=zhen_en_train_token2id)

# zhen_train_loader = torch.utils.data.DataLoader(dataset=zhen_train_dataset,
#                                batch_size=BATCH_SIZE,
#                                collate_fn=lambda x, max_sentence_length=MAX_SENTENCE_LENGTH: translation_collate(x, MAX_SENTENCE_LENGTH),
#                                shuffle=False)

## Part 2: Evaluation Metric

We use BLEU as the evaluation metric. Specifically, we focus on the corpus-level BLEU function. 

The code for BLEU is taken from https://github.com/mjpost/sacreBLEU/blob/master/sacrebleu.py#L1022-L1080

In [44]:
! pip3 install sacrebleu

You are using pip version 18.0, however version 18.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [126]:
# import sacrebleu

In [127]:
# TODO

## Part 3: Beam Search Algorithm

In this section, we implement the Beam Search algorithm in Pytorch.

## Part 4: Model

1. Recurrent neural network based encoder-decoder without attention
2. Recurrent neural network based encoder-decoder with attention
2. Replace the recurrent encoder with either convolutional or self-attention based encoder.

#### Loss Function & Evaluation

In [46]:
# reconstruction loss = binary cross entropy between two (vocab_size x 1) vectors
# used during training, since we can compare the real Y and and the generated Y
# still at each time step of the decoder, we compare up to and including
# the real t-th token and the generated t-th, then optimize

def loss_function(y_hat, y):
    
    """Takes as input;
    - y: correct "log-softmax"(binary vector) that represents the correct t-th token in the target sentence,
                 (vocab_size x 1) vector
    - y_hat: predicted LogSoftmax for the predicted t-th token in the target sentence.
             (vocab_size x 1) vector
    Returns;
    - NLL Loss in training time"""
#     y_hat = torch.log(y_hat) # log softmax
    loss = nn.functional.binary_cross_entropy(y_hat,y)
    
    return loss
    

# generation/inference time - validation loss = BLEU

def compute_BLEU(corpus_hat,corpus):
    ## TODO
    return None


#### Beam Search

### Beam Search

In [48]:
torch.FloatTensor([3,4,2,7,5,3,2]).topk(3)

(tensor([7., 5., 4.]), tensor([3, 4, 1]))

In [49]:
BATCH_SIZE = 32

class BeamSearch(nn.Module):
    
    """network that conducts beam search over the outputs of
     any translator network. The translator networks that can 
     be passed are:
     
     - Translate (for RNN-enc-dec),
     - AttnTranslate (for RNN-enc-dec with attention),
     - CNNtranslate (for CNN-encoder based translation).
     
     The translation networks take care of the encoder-decoder
     choices specific to each task. Please see in below sections."""

    def __init__(self, translator_network, beam_size):
        super().__init__()
        # translator network that returns the logsoftmax
        # over vocabulary size:(vocab_size, 1)
        self.translator_network = translator_network
        self.beam_size = beam_size
        
    def init_search_tree(self, batch_size):
        beam_size = self.beam_size
        self.search_tree = torch.empty(batch_size, beam_size, 1)
        return self
    
    def init_score_tree(self, batch_size):
        beam_size = self.beam_size
        search_tree = self.search_tree
        self.score_tree = torch.zeros(search_tree.size())
        return self
    
    def forward(source_sentence, source_mask, source_lengths,
                target_sentence, target_mask, target_lengths):
        
        self.init_search_tree(BATCH_SIZE)
        self.init_score_tree(BATCH_SIZE)
        
        # at each time step the decoder will give us the logsoftmax
        # of one token (batch_size, vocab_size). 
        output = model(source_sentence, target_sentence,source_mask, 
                       target_mask, source_lengths,target_lengths)
        
        # for each sentence in the batch we get the top k predictions
        # for each token and append it to the search and score trees. 
        for i in range(BATCH_SIZE):
            beam = output[i].topk(beam_size) # (token scores, token indices)
            # cat instead
            self.search_tree[i] = self.search_tree.cat(beam[1]) # cat the indices to the search tree
            self.score_tree[i,:] = beam[0] # append the scores to the score tree 
        
        # we will sum the logs 
        
        

### 2.1: RNN-based Encoder-Decoder without Attention

In [50]:
batch_size = 32

In [54]:
def convert_to_softmax(tensor_of_indices,
                       batch_size,
                       vocab_size = len(zhen_en_train_token2id)):
    """
    - takes as input a time_step vector of the batch (t-th token of each sentence in the batch)
      size: (batch_size, 1)
    - converts it to softmax of (batch_size, vocab_size)
    """
    index_tensor_ = tensor_of_indices.view(-1,1).long()
        
    one_hot = torch.FloatTensor(batch_size, vocab_size).zero_()
    one_hot.scatter_(1, index_tensor_.detach().cpu(), 1)
    
    return one_hot

### Fully self-attention Translation System / Transformer

In [4]:
%run ../models/transformer/EncoderDecoder.py
%run ../models/transformer/transformer_utils.py

In [5]:
tmp_model = make_model(10, 10, 2)

NameError: name 'MultiHeadedAttention' is not defined